In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = "/content/drive/MyDrive/xray_bone_fracture/"

training_path = folder_path + "train"
testing_path = folder_path + "val"


In [ ]:
training_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

testing_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [ ]:
training_generator = training_datagen.flow_from_directory(
    training_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
testing_generator = testing_datagen.flow_from_directory(
    testing_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.summary()

43941136/43941136 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional  (None, 7, 7, 1536)        10783535  
 )                                                               
                                                                 
 gaussian_noise (GaussianNo  (None, 7, 7, 1536)        0         
 ise)                                                            
                                                                 
 global_average_pooling2d (  (None, 1536)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 batch_normalization (Batch  (None, 512)               20

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','Precision','Recall','AUC'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
lrp=ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
call=[checkpoint,lrp]
history = model.fit(
    training_generator,
    epochs=20,
    validation_data=testing_generator,
    steps_per_epoch= 50,
    callbacks=call
    )

Epoch 1/20
50/50 [==============================] - ETA: 0s - loss: 4.4038 - accuracy: 0.5750 - precision: 0.5795 - recall: 0.5152 - auc: 0.5689
Epoch 1: val_accuracy improved from -inf to 0.47333, saving model to best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


50/50 [==============================] - 308s 6s/step - loss: 4.4038 - accuracy: 0.5750 - precision: 0.5795 - recall: 0.5152 - auc: 0.5689 - val_loss: 1.4993 - val_accuracy: 0.4733 - val_precision: 0.4275 - val_recall: 0.9333 - val_auc: 0.6459 - lr: 0.0010
Epoch 2/20
50/50 [==============================] - ETA: 0s - loss: 4.5818 - accuracy: 0.5750 - precision: 0.5647 - recall: 0.5000 - auc: 0.5615
Epoch 2: val_accuracy did not improve from 0.47333
50/50 [==============================] - 48s 968ms/step - loss: 4.5818 - accuracy: 0.5750 - precision: 0.5647 - recall: 0.5000 - auc: 0.5615 - val_loss: 2.5002 - val_accuracy: 0.3833 - val_precision: 0.3367 - val_recall: 0.5583 - val_auc: 0.3450 - lr: 0.0010
Epoch 3/20
50/50 [==============================] - ETA: 0s - loss: 5.2254 - accuracy: 0.5450 - precision: 0.5385 - recall: 0.5000 - auc: 0.5475
Epoch 3: val_accuracy improved from 0.47333 to 0.57000, saving model to best_model.h5
50/50 [==============================] - 45s 911ms/step -

In [ ]:
model.evaluate(training_generator)

2216/2216 [==============================] - 1120s 505ms/step - loss: 1.5242 - accuracy: 0.6590 - precision: 0.6470 - recall: 0.6833 - auc: 0.7191


[1.5241954326629639,
 0.6590319275856018,
 0.6470080018043518,
 0.6833219528198242,
 0.719081461429596]

In [ ]:
model.evaluate(testing_generator)

150/150 [==============================] - 12s 82ms/step - loss: 2.3246 - accuracy: 0.5800 - precision: 0.4400 - recall: 0.1833 - auc: 0.5534


[2.3245773315429688,
 0.5799999833106995,
 0.4399999976158142,
 0.18333333730697632,
 0.5533796548843384]